## BIRCH Algorithm Implementation

In [27]:
# Add Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.cluster import AgglomerativeClustering

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import Birch as SklearnBIRCH
from sklearn.metrics import adjusted_rand_score

In [28]:
dataset_path = "./../datasets"

iris_dataset_path = dataset_path + "/iris.csv"                                         
ai_global_index_path = dataset_path + "/AI_index_db.csv"
global_earthquake_data_path = dataset_path + "/earthquakes.csv"

datasets = {
    "iris": iris_df,
    "ai_global_index": ai_global_index_df,
    "global_earthquake": global_earthquake_data_df
}


In [4]:
iris_df = pd.read_csv(iris_dataset_path)
ai_global_index_df = pd.read_csv(ai_global_index_path)
global_earthquake_data_df = pd.read_csv(global_earthquake_data_path)

### BIRCH Implementation (Based on our Algorithm - see report/Part-1.pdf)

In [59]:
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

class ClusteringFeature:
    def __init__(self, point):
        self.N = 1  # Number of points
        self.LS = np.array(point)  # Linear sum of points
        self.SS = np.array(point) ** 2  # Squared sum of points

    def add_point(self, point):
        self.N += 1
        self.LS += np.array(point)
        self.SS += np.array(point) ** 2

    def centroid(self):
        return self.LS / self.N
    
    def radius(self):
        return np.linalg.norm(self.LS / self.N)

    def can_absorb(self, point, threshold):
        new_centroid = (self.LS + np.array(point)) / (self.N + 1)
        distance = euclidean(new_centroid, point)
        return distance <= threshold


class CFNode:
    def __init__(self, branching_factor):
        self.branching_factor = branching_factor
        self.children = []
        self.cf = None

    def is_leaf(self):
        return len(self.children) == 0

    def insert(self, point, threshold):
        if self.is_leaf():
            if self.cf is None:
                self.cf = ClusteringFeature(point)
            elif self.cf.can_absorb(point, threshold):
                self.cf.add_point(point)
            else:
                self.split(point, threshold)
        else:
            # Filter out nodes with `cf` as None
            valid_children = [child for child in self.children if child.cf is not None]

            if valid_children:
                closest_child = min(valid_children, key=lambda child: euclidean(child.cf.centroid(), point))
                closest_child.insert(point, threshold)

    def split(self, point, threshold):
        points = [self.cf.centroid(), point]
        kmeans = KMeans(n_clusters=2, n_init=10).fit(points)

        new_node1 = CFNode(self.branching_factor)
        new_node2 = CFNode(self.branching_factor)

        # Assign points to the new nodes
        for i, cluster_id in enumerate(kmeans.labels_):
            target_node = new_node1 if cluster_id == 0 else new_node2
            target_node.cf = ClusteringFeature(points[i])

        # Replace current node with child nodes
        self.children = [new_node1, new_node2]
        self.cf = None  # Reset the CF for this node


class CFTree:
    def __init__(self, threshold, branching_factor):
        self.threshold = threshold
        self.branching_factor = branching_factor
        self.root = CFNode(branching_factor)

    def insert(self, point):
        self.root.insert(point, self.threshold)
        
    def get_leaves(self, node=None):
        if node is None:
            node = self.root
        if node.is_leaf():
            return [node.cf]
        leaves = []
        for child in node.children:
            leaves.extend(self.get_leaves(child))
        return leaves


def birch_clustering(data, threshold, branching_factor, n_clusters):
    # Phase 1: Build the CF Tree
    cf_tree = CFTree(threshold, branching_factor)
    for point in data:
        cf_tree.insert(point)

    # Phase 2: Cluster the Leaves
    leaves = cf_tree.get_leaves()
    leaf_centroids = [leaf.centroid() for leaf in leaves]

    if len(leaf_centroids) < 2:
        print("Warning: Only one leaf node found. Cannot perform clustering.")
        return cf_tree, leaf_centroids

    # Agglomerative clustering on leaf centroids
    agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)
    cluster_labels = agg_clustering.fit_predict(leaf_centroids)

    # Compute final cluster centroids
    cluster_centroids = []
    for i in range(n_clusters):
        cluster_points = [leaf_centroids[j] for j in range(len(leaf_centroids)) if cluster_labels[j] == i]
        cluster_centroids.append(np.mean(cluster_points, axis=0))

    return cf_tree, cluster_centroids


# Custom function to assign cluster to each data point
def assign_clusters(data_points, centroids):
    distances = np.linalg.norm(data_points[:, np.newaxis] - centroids, axis=2)
    cluster_labels = np.argmin(distances, axis=1)
    return cluster_labels


# Example Usage
if __name__ == "__main__":
    # Sample dataset
    data = np.array([
        [1, 2], [1, 4], [1, 0],
        [10, 2], [10, 4], [10, 0]
    ])

    # Parameters
    threshold = 0.5  # Reduced threshold to ensure splitting
    branching_factor = 3  
    n_clusters = 2  

    # Run BIRCH clustering
    cf_tree, cluster_centroids = birch_clustering(data, threshold, branching_factor, n_clusters)

    # Assign cluster to each data point
    custom_labels = assign_clusters(data, cluster_centroids)

    print("\nCustom BIRCH Centroids for ai_global_index:")
    for i, centroid in enumerate(cluster_centroids):
        print(f"Cluster {i + 1}: {centroid}")

    



Custom BIRCH Centroids for ai_global_index:
Cluster 1: [1. 2.]
Cluster 2: [10.  2.]

Custom BIRCH Labels for ai_global_index:
[0 0 0 1 1 1]


D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [65]:
results = {}

for name, df in datasets.items():
    df = df.dropna()

    # Extract numerical features
    X = df.select_dtypes(include=[np.number]).values

    # Normalize the data
    X = StandardScaler().fit_transform(X)

    # Run the custom BIRCH implementation
    threshold = 0.01  # Adjust as needed
    branching_factor = 10  # Adjust as needed
    n_clusters = 3  # Adjust as needed

    cf_tree, custom_centroids = birch_clustering(X, threshold, branching_factor, n_clusters)
    custom_labels = np.zeros(len(X))  # Placeholder for custom labels
    # Assign labels based on closest centroid (if needed)
    for i, point in enumerate(X):
        distances = [euclidean(point, centroid) for centroid in custom_centroids]
        custom_labels[i] = np.argmin(distances)

    print(f"Custom BIRCH Centroids for {name}:")
    for i, centroid in enumerate(custom_centroids):
        print(f"Cluster {i + 1}: {centroid}")
    print(f"\nCustom BIRCH Labels for {name}: {custom_labels}")
    #print(custom_labels)

    # Run the sklearn BIRCH implementation
    sklearn_birch = SklearnBIRCH(threshold=threshold, branching_factor=branching_factor, n_clusters=n_clusters)
    sklearn_labels = sklearn_birch.fit_predict(X)

    print(f"Sklearn BIRCH Labels for {name}: {sklearn_labels}")

    # Compare the results using Adjusted Rand Index (ARI)
    ari_score = adjusted_rand_score(custom_labels, sklearn_labels)
    results[name] = ari_score
    print(f"Adjusted Rand Index (ARI) for {name}: {ari_score}")

# Store results
results = pd.Series(results)
results.to_csv("./../results/birch_comparison.csv", header=False)

D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than availabl

Custom BIRCH Centroids for iris:
Cluster 1: [-1.44593659  0.22214685 -1.3412724  -1.31297673]
Cluster 2: [-0.90068117  1.03205722 -1.3412724  -1.31297673]
Cluster 3: [-1.14301691 -0.1249576  -1.3412724  -1.31297673]

Custom BIRCH Labels for iris: [1. 2. 0. 0. 1. 1. 0. 1. 2. 2. 1. 1. 2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 2. 1. 1. 1. 0. 0. 1. 1. 1. 2. 0. 1. 2. 0. 1. 1. 2. 0. 1. 1. 2. 1. 0.
 1. 1. 1. 1. 1. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 1. 1. 2. 2. 2. 2. 1. 2. 1. 2. 2.
 1. 2. 2. 2. 1. 1. 2. 2. 2. 2. 2. 1. 2. 2. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 2. 2. 2. 1. 2.]
Sklearn BIRCH Labels for iris: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 1 1 1 1 1 1 1 1 0 0 0 2 0 2 0 2 0 2 2 0 2 0 2 0 2 2 2 2 0 0 0 0
 0 0 0 0 0 2 2 2 2 0 2 0 0 2 2 2 2 0 2 2 2 2 2 0 2 2 0 0 0 0 0 0 2 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 

D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
